# Homework problem

In an office building there are 11 floors on which 30 tenants are to be located. Each tenant has given space requirements that may vary with the floor to which he is assigned. If tenant j is assigned to floor i, he needs $a_{ij}$ $m^2$ of floor space Floor i is assumed to have an area of $b_i$ $m^2$. Suppose that each m^2 of space on floor i that is rented yields $c_i$ $/month for the company. To maximize the revenue which tenant should be assigned to which floor?

### Discrete Value Representation

You may use discrete value representation like the following:

| 1 | 2 | 3 | ... | 30 |
|---|---|---|-----|----|
| 4 | 5 | 1 | ... | 5  |

